In [1]:
# 1.卷积神经网络
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
# 2.hyper parameters
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [5]:
# 3.MINIST Dataset
train_dataset = dsets.MNIST(root='../data/',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)
test_dataset = dsets.MNIST(root='../data/',
                          train=False,
                          transform=transforms.ToTensor())

In [6]:
# 4.Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


In [7]:
# 5.CNN Model(2 conv layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,16,kernel_size=5,padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32,kernel_size=5,padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32,10)
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        return out
cnn = CNN()
        

In [8]:
cnn

CNN (
  (layer1): Sequential (
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU ()
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (layer2): Sequential (
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU ()
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (fc): Linear (1568 -> 10)
)

In [9]:
# 6.Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(),lr =learning_rate)


In [10]:
# 7.train the mosel
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)
        
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
        if (i+1)%100 == 0:
            print('Epoch [%d/%d],Iter [%d/%d]Loss: %.4f'
                 %(epoch+1,num_epochs,i+1,
                  len(train_dataset)//batch_size,loss.data[0]))
    

Epoch [1/5],Iter [100/600]Loss: 0.1964
Epoch [1/5],Iter [200/600]Loss: 0.0626
Epoch [1/5],Iter [300/600]Loss: 0.0821
Epoch [1/5],Iter [400/600]Loss: 0.0738
Epoch [1/5],Iter [500/600]Loss: 0.0633
Epoch [1/5],Iter [600/600]Loss: 0.0356
Epoch [2/5],Iter [100/600]Loss: 0.1140
Epoch [2/5],Iter [200/600]Loss: 0.0546
Epoch [2/5],Iter [300/600]Loss: 0.0450
Epoch [2/5],Iter [400/600]Loss: 0.0135
Epoch [2/5],Iter [500/600]Loss: 0.0205
Epoch [2/5],Iter [600/600]Loss: 0.0943
Epoch [3/5],Iter [100/600]Loss: 0.0220
Epoch [3/5],Iter [200/600]Loss: 0.0267
Epoch [3/5],Iter [300/600]Loss: 0.0397
Epoch [3/5],Iter [400/600]Loss: 0.0306
Epoch [3/5],Iter [500/600]Loss: 0.0022
Epoch [3/5],Iter [600/600]Loss: 0.0526
Epoch [4/5],Iter [100/600]Loss: 0.0264
Epoch [4/5],Iter [200/600]Loss: 0.0278
Epoch [4/5],Iter [300/600]Loss: 0.0082
Epoch [4/5],Iter [400/600]Loss: 0.0115
Epoch [4/5],Iter [500/600]Loss: 0.0110
Epoch [4/5],Iter [600/600]Loss: 0.0613
Epoch [5/5],Iter [100/600]Loss: 0.0056
Epoch [5/5],Iter [200/600

In [11]:
# 8.Test the Model
cnn.eval()  # Change model to 'eval' mode (BN uses moving mean/var). 归一化
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images)
    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98 %


In [12]:
# 9.Save the Trained Model
torch.save(cnn.state_dict(),'cnn.pkl')